In [143]:
import pandas as pd
import nltk
import numpy as np
import seaborn as sns
import matplotlib as plt
from nltk.tokenize import word_tokenize
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from nltk.corpus import stopwords
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import RandomizedSearchCV, StratifiedKFold

In [144]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [145]:
df_train = pd.read_csv("drive/MyDrive/train.csv")

# df_train = df_train.sample(frac=1, random_state=42)


df_test = pd.read_csv("drive/MyDrive/test.csv")

df_train['sentimiento'] = df_train['sentimiento'].map({'positivo': 1, 'negativo': 0})

In [146]:
RUTA_GUARDADO = "RED.csv"

In [147]:
def remove_in_words(lista_textos):
  texto_total = []
  # for parrafos in lista_textos:
  #   parrafo = []
  #   # for palabra in parrafos.split():
  #   #   palabras = []
  #   #   fo

  #   parrafo = ' '.join(parrafo)
  #   texto_total.append(parrafo)
  return texto_total

In [148]:
def pre_process(lista_corpus):
  return lista_corpus


In [149]:
texto = ["ho with the standnard compression stocking, 20-30, stock #123456. Excelent supoort,"]

In [150]:
print(texto)
print("---")
texto_limpio = remove_in_words(texto)
print(texto_limpio)
print("---")
texto_limpio = pre_process(texto)
print(texto_limpio)
print("---")


['ho with the standnard compression stocking, 20-30, stock #123456. Excelent supoort,']
---
[]
---
['ho with the standnard compression stocking, 20-30, stock #123456. Excelent supoort,']
---


In [151]:
df_train.head(5)

,ID,review_es,sentimiento
0,0,Uno de los otros críticos ha mencionado que de...,1
1,1,Una pequeña pequeña producción.La técnica de f...,1
2,2,Pensé que esta era una manera maravillosa de p...,1
3,3,"Básicamente, hay una familia donde un niño peq...",0
4,4,"El ""amor en el tiempo"" de Petter Mattei es una...",1


In [152]:
total_reviews = df_train['review_es'].astype(str)
total_labels = df_train['sentimiento'].values

print(total_reviews)
print(total_labels)

0        Uno de los otros críticos ha mencionado que de...
1        Una pequeña pequeña producción.La técnica de f...
2        Pensé que esta era una manera maravillosa de p...
3        Básicamente, hay una familia donde un niño peq...
4        El "amor en el tiempo" de Petter Mattei es una...
                               ...                        
49995    Pensé que esta película hizo un buen trabajo a...
49996    Mala parcela, mal diálogo, mala actuación, dir...
49997    Soy católica enseñada en escuelas primarias pa...
49998    Voy a tener que estar en desacuerdo con el com...
49999    Nadie espera que las películas de Star Trek se...
Name: review_es, Length: 50000, dtype: object
[1 1 1 ... 0 0 0]


In [153]:
import matplotlib.pyplot as plt

def plot_training_history(history):
    # Graficar la precisión del entrenamiento y la validación
    plt.figure(figsize=(12, 4))

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Entrenamiento')
    plt.plot(history.history['val_accuracy'], label='Validación')
    plt.title('Precisión del Modelo')
    plt.xlabel('Épocas')
    plt.ylabel('Precisión')
    plt.legend()

    # Graficar la pérdida del entrenamiento y la validación
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Entrenamiento')
    plt.plot(history.history['val_loss'], label='Validación')
    plt.title('Pérdida del Modelo')
    plt.xlabel('Épocas')
    plt.ylabel('Pérdida')
    plt.legend()

    plt.show()


In [154]:
Y_data = df_train.sentimiento

In [155]:
from tensorflow.keras.preprocessing.text import Tokenizer


In [156]:
max_words = 5000
max_len = 100

In [157]:
tokenizer_obj = Tokenizer(num_words = max_words)
tokenizer_obj.fit_on_texts(total_reviews)

In [158]:
X_data_tokens = tokenizer_obj.texts_to_sequences(total_reviews)

In [159]:
X_data_count = tokenizer_obj.sequences_to_matrix(X_data_tokens, mode = 'count')

In [160]:
X_train_count, X_test_count, y_train_count, y_test_count = train_test_split(X_data_count, Y_data, test_size = 0.3, random_state = 123)

In [161]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [162]:
model_count = Sequential()
model_count.add(Dense(100, activation='relu', input_shape=(5000,)))
model_count.add(Dense(25, activation='relu'))
model_count.add(Dense(1, activation='sigmoid'))


model_count.compile(loss = "binary_crossentropy", optimizer = 'adam', metrics = ['accuracy'])



In [179]:
history_count = model_count.fit(X_train_count, y_train_count, batch_size = 256, epochs = 15, validation_split=0.3)



Epoch 1/15
96/96 [==============================] - 4s 38ms/step - loss: 0.2520 - accuracy: 0.9014 - val_loss: 0.3485 - val_accuracy: 0.8651
Epoch 2/15
96/96 [==============================] - 3s 28ms/step - loss: 0.2060 - accuracy: 0.9229 - val_loss: 0.3745 - val_accuracy: 0.8612
Epoch 3/15
96/96 [==============================] - 3s 28ms/step - loss: 0.1565 - accuracy: 0.9437 - val_loss: 0.4049 - val_accuracy: 0.8596
Epoch 4/15
96/96 [==============================] - 3s 35ms/step - loss: 0.1000 - accuracy: 0.9678 - val_loss: 0.4526 - val_accuracy: 0.8547
Epoch 5/15
96/96 [==============================] - 4s 42ms/step - loss: 0.0549 - accuracy: 0.9866 - val_loss: 0.5333 - val_accuracy: 0.8562
Epoch 6/15
96/96 [==============================] - 3s 26ms/step - loss: 0.0243 - accuracy: 0.9960 - val_loss: 0.6188 - val_accuracy: 0.8516
Epoch 7/15
96/96 [==============================] - 3s 28ms/step - loss: 0.0197 - accuracy: 0.9970 - val_loss: 0.6623 - val_accuracy: 0.8553
Epoch 8/15
96

In [180]:
test_loss, test_acc = model_count.evaluate(X_test_count, y_test_count)
print('')
print(test_loss, test_acc)

ValueError: Data cardinality is ambiguous:
  x sizes: 8599
  y sizes: 15000
Make sure all arrays contain the same number of samples.

In [ ]:
plot_training_history(history_count)


In [181]:
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix, f1_score, precision_score, recall_score, accuracy_score

In [ ]:
def imprimir_resultados(y, y_pred):

    labels = [0, 1]

    f1 = f1_score(y, y_pred, pos_label=0, labels=labels)
    precision = precision_score(y, y_pred, pos_label=0, labels=labels)
    recall = recall_score(y, y_pred, pos_label=0, labels=labels)
    accuracy = accuracy_score(y, y_pred)

    cm = confusion_matrix(y_test_, y_pred)

    ConfusionMatrixDisplay(confusion_matrix=cm).plot()

    print(f"F1: {f1}")
    print(f"Precision: {precision}")
    print(f"Recall: {recall}")
    print(f"Accuracy: {accuracy}")

In [ ]:
import numpy as np
X_train_tokens = tokenizer_obj.texts_to_sequences(df_test["review_es"])
X_train_count = tokenizer_obj.sequences_to_matrix(X_train_tokens, mode = 'count')
y_pred = model_count.predict(X_test_count)
y_pred = np.round(y_pred)


In [ ]:
imprimir_resultados(y_train_count, y_pred)

In [167]:
import numpy as np
X_test_tokens = tokenizer_obj.texts_to_sequences(df_test["review_es"])
X_test_count = tokenizer_obj.sequences_to_matrix(X_test_tokens, mode = 'count')
y_pred = model_count.predict(X_test_count)
y_pred = np.round(y_pred)


269/269 [==============================] - 1s 4ms/step


In [168]:
y_pred

array([[0.],
       [0.],
       [0.],
       ...,
       [0.],
       [0.],
       [0.]], dtype=float32)

In [175]:
y_pred = np.where(y_pred == 0, "negativo", "positivo")

In [176]:
def guardar_resultado_en_csv(prediccion):
    # Flatten the prediccion array to 1D
    prediccion = prediccion.flatten()

    df_predicciones = pd.DataFrame({"ID": df_test["ID"], "prediccion": prediccion})

    df_predicciones.to_csv(RUTA_GUARDADO, header=["ID", "sentimiento"], index=False)

In [177]:
guardar_resultado_en_csv(y_pred)